In [180]:
import pandas as pd
import pmdarima as pm
from sklearn.metrics import mean_squared_error



In [286]:

df=pd.read_csv('SAP successfactors.csv', parse_dates=['Year'], index_col='Year')
arima_fit=pm.auto_arima(df['Total Talents'],error_action='ignore',suppress_warnings=True,seasonal=False)

arima_fit.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   12
Model:               SARIMAX(0, 1, 0)   Log Likelihood                 -28.209
Date:                Wed, 10 May 2023   AIC                             60.417
Time:                        13:59:45   BIC                             61.213
Sample:                    01-01-2011   HQIC                            59.915
                         - 01-01-2022                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      7.5455      1.036      7.284      0.000       5.515       9.576
sigma2         9.8843      6.788      1.456      0.145      -3.420      23.189
===================================================================================
Ljung-Box (L1) (Q):                   1.77   Jarque-Bera (JB):                 0.81
Prob(Q):                              0.18   Prob(JB):                         0.67
Heteroskedasticity (H):               0.41   Skew:                             0.39
Prob(H) (two-sided):                  0.40   Kurtosis:                         1.92
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [287]:
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
data = df

ts = pd.Series(data['Total Talents'], index=pd.to_datetime(data.index))

train_size = int(len(ts) * 0.8)  # 80% of the data for training
train, test = ts[:train_size], ts[train_size:]
model = ARIMA(train, order=arima_fit.order)
results = model.fit()
predictions = results.predict(start=len(train), end=len(ts)-1, typ='levels')

expected = test.values
predicted=np.round(predictions.values)
rmse = np.sqrt(np.mean((predicted - expected)**2))
print("RMSE:", rmse)

print(expected,predicted)



RMSE: 13.73559851869101
[76 83 89] [70. 70. 70.]


c:\Users\haffi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
c:\Users\haffi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
c:\Users\haffi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
c:\Users\haffi\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['typ']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.w

In [288]:
forecast_horizon = 5
fc, conf = arima_fit.predict(n_periods=forecast_horizon, return_conf_int=True)


print(fc, conf)



2023-01-01     96.545455
2024-01-01    104.090909
2025-01-01    111.636364
2026-01-01    119.181818
2027-01-01    126.727273
Freq: AS-JAN, dtype: float64 [[ 90.38346445 102.70744464]
 [ 95.37653913 112.80527905]
 [100.96348372 122.30924355]
 [106.857838   131.50579837]
 [112.948644   140.50590145]]


In [289]:
# Create a datetime index for the forecast dates
start_date = pd.to_datetime(df.index.max(), format='%Y') + pd.DateOffset(years=1)
freq = 'YS'
fc_array = np.round(np.array(fc))
conf=np.round(conf)
forecast_dates = pd.date_range(start=start_date, periods=forecast_horizon, freq=freq)
forecast_df = pd.DataFrame({'Total Talents': fc_array, 'Lower Bound': conf[:, 0], 'Upper Bound': conf[:, 1]}, index=forecast_dates)

# Print the forecasted values with the year as the index
print(forecast_df)

            Total Talents  Lower Bound  Upper Bound
2023-01-01           97.0         90.0        103.0
2024-01-01          104.0         95.0        113.0
2025-01-01          112.0        101.0        122.0
2026-01-01          119.0        107.0        132.0
2027-01-01          127.0        113.0        141.0


In [290]:
forecast_df.to_csv('forecast.csv')
